https://www.kaggle.com/code/ndujar/kernel-music-reccommender

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import seaborn as sns

In [2]:
data = pd.read_csv("../feature_extraction/dataForHarmonicAudio.csv",usecols=lambda column: column != 'filename')

In [3]:
mappingGenresToIntegers = {
    'pop': 0,
    'metal': 1,
    'disco': 2,
    'blues': 3,
    'reggae': 4,
    'classical': 5,
    'rock': 6,
    'hiphop': 7,
    'country': 8,
    'jazz': 9
}


dfWithGenresMappedToInts = data.copy()  # Create a copy of the original DataFrame
dfWithGenresMappedToInts['genre'] = dfWithGenresMappedToInts['genre'].map(mappingGenresToIntegers)

dfWithGenresMappedToInts.head()

,chroma_C,chroma_Csharp,chroma_D,chroma_Dsharp,chroma_E,chroma_F,chroma_Fsharp,chroma_G,chroma_Gsharp,chroma_A,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,genre
0,0.391512,0.429697,0.524729,0.385698,0.366340,0.433856,0.445597,0.593311,0.438685,0.413015,...,0.164212,2.695709,1.318642,0.632315,0.957001,-1.566437,6.275709,7.807974,6.363476,0
1,0.544175,0.420254,0.424198,0.399732,0.300949,0.312941,0.363381,0.445987,0.342582,0.325424,...,5.130442,6.827752,3.071533,3.366904,-2.422404,-2.135466,-3.336569,-0.564991,-3.893568,0
2,0.344232,0.477342,0.461315,0.383294,0.430534,0.360271,0.595679,0.406584,0.354102,0.464613,...,1.111607,-2.048577,-4.366745,-3.732610,-3.398216,2.417332,1.289159,7.578645,4.909898,0
3,0.392071,0.428158,0.523789,0.385555,0.366227,0.433113,0.444056,0.591267,0.437267,0.413407,...,0.235207,2.670471,1.338333,0.699207,1.112011,-1.565463,6.195435,7.648764,6.243628,0
4,0.396111,0.442692,0.408399,0.486187,0.353345,0.326342,0.372000,0.300399,0.334868,0.334352,...,-0.269821,-1.642580,-4.781828,-3.302547,-4.714705,-3.559506,-3.702941,-4.042264,0.333003,0


In [4]:
X = dfWithGenresMappedToInts.iloc[:, :-1] 
y = dfWithGenresMappedToInts.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)

In [5]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
!pip install catboost

  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/c8/c0/83de2c76f5b49d0fd71912facf3ed4fe2db06bf225d526883887688b6156/catboost-1.2.3-cp311-cp311-macosx_11_0_universal2.whl.metadata
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/00/be/d59db2d1d52697c6adc9eacaf50e8965b6345cc143f671e1ed068818d5cf/graphviz-0.20.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.2/26.2 MB 16.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.0 MB/s eta 0:00:00


In [9]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification

catboost = CatBoostClassifier()

param_grid = {
    'iterations': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'loss_function': ['MultiClass'],
    'eval_metric': ['AUC']
}

grid_search = GridSearchCV(estimator=catboost, param_grid=param_grid, cv=5)

grid_search.fit(X_train, y_train)

0:	total: 7.24ms	remaining: 717ms
1:	total: 16.2ms	remaining: 795ms
2:	total: 22.6ms	remaining: 730ms
3:	total: 26.7ms	remaining: 641ms
4:	total: 31.2ms	remaining: 593ms
5:	total: 35.8ms	remaining: 561ms
6:	total: 39.7ms	remaining: 527ms
7:	total: 43.7ms	remaining: 502ms
8:	total: 48.6ms	remaining: 492ms
9:	total: 52.5ms	remaining: 473ms
10:	total: 56.4ms	remaining: 456ms
11:	total: 62.7ms	remaining: 460ms
12:	total: 67.6ms	remaining: 453ms
13:	total: 71.7ms	remaining: 440ms
14:	total: 76ms	remaining: 431ms
15:	total: 81.4ms	remaining: 427ms
16:	total: 85.4ms	remaining: 417ms
17:	total: 89.7ms	remaining: 409ms
18:	total: 94.3ms	remaining: 402ms
19:	total: 98.9ms	remaining: 396ms
20:	total: 103ms	remaining: 386ms
21:	total: 107ms	remaining: 379ms
22:	total: 110ms	remaining: 370ms
23:	total: 115ms	remaining: 364ms
24:	total: 119ms	remaining: 358ms
25:	total: 124ms	remaining: 352ms
26:	total: 130ms	remaining: 352ms
27:	total: 134ms	remaining: 345ms
28:	total: 139ms	remaining: 340ms
29:	to

54:	total: 422ms	remaining: 346ms
55:	total: 437ms	remaining: 344ms
56:	total: 460ms	remaining: 347ms
57:	total: 466ms	remaining: 337ms
58:	total: 470ms	remaining: 327ms
59:	total: 476ms	remaining: 317ms
60:	total: 481ms	remaining: 307ms
61:	total: 492ms	remaining: 302ms
62:	total: 497ms	remaining: 292ms
63:	total: 502ms	remaining: 282ms
64:	total: 506ms	remaining: 273ms
65:	total: 513ms	remaining: 264ms
66:	total: 517ms	remaining: 255ms
67:	total: 522ms	remaining: 246ms
68:	total: 529ms	remaining: 238ms
69:	total: 535ms	remaining: 229ms
70:	total: 541ms	remaining: 221ms
71:	total: 546ms	remaining: 212ms
72:	total: 551ms	remaining: 204ms
73:	total: 556ms	remaining: 195ms
74:	total: 564ms	remaining: 188ms
75:	total: 571ms	remaining: 180ms
76:	total: 578ms	remaining: 173ms
77:	total: 583ms	remaining: 164ms
78:	total: 589ms	remaining: 156ms
79:	total: 596ms	remaining: 149ms
80:	total: 601ms	remaining: 141ms
81:	total: 607ms	remaining: 133ms
82:	total: 613ms	remaining: 126ms
83:	total: 618

99:	total: 689ms	remaining: 0us
0:	total: 8.66ms	remaining: 858ms
1:	total: 14.6ms	remaining: 717ms
2:	total: 21.4ms	remaining: 691ms
3:	total: 28.8ms	remaining: 690ms
4:	total: 33.7ms	remaining: 640ms
5:	total: 39.2ms	remaining: 615ms
6:	total: 47.1ms	remaining: 626ms
7:	total: 53.3ms	remaining: 613ms
8:	total: 61.6ms	remaining: 623ms
9:	total: 67.1ms	remaining: 604ms
10:	total: 74.7ms	remaining: 604ms
11:	total: 79.4ms	remaining: 583ms
12:	total: 87.8ms	remaining: 587ms
13:	total: 98.1ms	remaining: 603ms
14:	total: 103ms	remaining: 581ms
15:	total: 109ms	remaining: 570ms
16:	total: 115ms	remaining: 564ms
17:	total: 121ms	remaining: 550ms
18:	total: 128ms	remaining: 544ms
19:	total: 132ms	remaining: 528ms
20:	total: 136ms	remaining: 511ms
21:	total: 141ms	remaining: 499ms
22:	total: 145ms	remaining: 487ms
23:	total: 151ms	remaining: 478ms
24:	total: 157ms	remaining: 472ms
25:	total: 162ms	remaining: 460ms
26:	total: 166ms	remaining: 448ms
27:	total: 170ms	remaining: 438ms
28:	total: 1

53:	total: 388ms	remaining: 330ms
54:	total: 394ms	remaining: 322ms
55:	total: 401ms	remaining: 315ms
56:	total: 406ms	remaining: 306ms
57:	total: 412ms	remaining: 299ms
58:	total: 418ms	remaining: 290ms
59:	total: 423ms	remaining: 282ms
60:	total: 427ms	remaining: 273ms
61:	total: 432ms	remaining: 265ms
62:	total: 436ms	remaining: 256ms
63:	total: 440ms	remaining: 248ms
64:	total: 445ms	remaining: 240ms
65:	total: 450ms	remaining: 232ms
66:	total: 456ms	remaining: 225ms
67:	total: 461ms	remaining: 217ms
68:	total: 468ms	remaining: 210ms
69:	total: 472ms	remaining: 202ms
70:	total: 477ms	remaining: 195ms
71:	total: 483ms	remaining: 188ms
72:	total: 488ms	remaining: 181ms
73:	total: 492ms	remaining: 173ms
74:	total: 498ms	remaining: 166ms
75:	total: 504ms	remaining: 159ms
76:	total: 510ms	remaining: 152ms
77:	total: 516ms	remaining: 145ms
78:	total: 523ms	remaining: 139ms
79:	total: 529ms	remaining: 132ms
80:	total: 532ms	remaining: 125ms
81:	total: 537ms	remaining: 118ms
82:	total: 547

93:	total: 519ms	remaining: 33.2ms
94:	total: 538ms	remaining: 28.3ms
95:	total: 543ms	remaining: 22.6ms
96:	total: 552ms	remaining: 17.1ms
97:	total: 560ms	remaining: 11.4ms
98:	total: 568ms	remaining: 5.73ms
99:	total: 573ms	remaining: 0us
0:	total: 10.4ms	remaining: 1.03s
1:	total: 19.1ms	remaining: 936ms
2:	total: 23.6ms	remaining: 764ms
3:	total: 31.6ms	remaining: 758ms
4:	total: 37.3ms	remaining: 709ms
5:	total: 42ms	remaining: 658ms
6:	total: 49.8ms	remaining: 662ms
7:	total: 55.3ms	remaining: 636ms
8:	total: 60.6ms	remaining: 613ms
9:	total: 65.7ms	remaining: 591ms
10:	total: 74.4ms	remaining: 602ms
11:	total: 81.5ms	remaining: 597ms
12:	total: 85.6ms	remaining: 573ms
13:	total: 91.4ms	remaining: 562ms
14:	total: 96.4ms	remaining: 546ms
15:	total: 101ms	remaining: 528ms
16:	total: 105ms	remaining: 511ms
17:	total: 111ms	remaining: 506ms
18:	total: 116ms	remaining: 496ms
19:	total: 121ms	remaining: 484ms
20:	total: 125ms	remaining: 470ms
21:	total: 131ms	remaining: 464ms
22:	tot

...

98:	total: 7.1s	remaining: 28.8s
99:	total: 7.18s	remaining: 28.7s
100:	total: 7.47s	remaining: 29.5s
101:	total: 7.63s	remaining: 29.8s
102:	total: 7.74s	remaining: 29.8s
103:	total: 7.81s	remaining: 29.8s
104:	total: 7.9s	remaining: 29.7s
105:	total: 7.98s	remaining: 29.7s
106:	total: 8.05s	remaining: 29.6s
107:	total: 8.14s	remaining: 29.5s
108:	total: 8.22s	remaining: 29.5s
109:	total: 8.32s	remaining: 29.5s
110:	total: 8.4s	remaining: 29.4s
111:	total: 8.48s	remaining: 29.4s
112:	total: 8.57s	remaining: 29.4s
113:	total: 8.65s	remaining: 29.3s
114:	total: 8.74s	remaining: 29.3s
115:	total: 8.82s	remaining: 29.2s
116:	total: 8.9s	remaining: 29.1s
117:	total: 8.97s	remaining: 29.1s
118:	total: 9.05s	remaining: 29s
119:	total: 9.13s	remaining: 28.9s
120:	total: 9.22s	remaining: 28.9s
121:	total: 9.29s	remaining: 28.8s
122:	total: 9.38s	remaining: 28.7s
123:	total: 9.45s	remaining: 28.7s
124:	total: 9.55s	remaining: 28.6s
125:	total: 9.63s	remaining: 28.6s
126:	total: 9.71s	remaining:

336:	total: 27.1s	remaining: 13.1s
337:	total: 27.2s	remaining: 13s
338:	total: 27.2s	remaining: 12.9s
339:	total: 27.3s	remaining: 12.8s
340:	total: 27.4s	remaining: 12.8s
341:	total: 27.5s	remaining: 12.7s
342:	total: 27.5s	remaining: 12.6s
343:	total: 27.6s	remaining: 12.5s
344:	total: 27.7s	remaining: 12.4s
345:	total: 27.8s	remaining: 12.4s
346:	total: 27.8s	remaining: 12.3s
347:	total: 27.9s	remaining: 12.2s
348:	total: 28s	remaining: 12.1s
349:	total: 28s	remaining: 12s
350:	total: 28.1s	remaining: 11.9s
351:	total: 28.1s	remaining: 11.8s
352:	total: 28.2s	remaining: 11.7s
353:	total: 28.3s	remaining: 11.7s
354:	total: 28.4s	remaining: 11.6s
355:	total: 28.5s	remaining: 11.5s
356:	total: 28.5s	remaining: 11.4s
357:	total: 28.6s	remaining: 11.3s
358:	total: 28.7s	remaining: 11.3s
359:	total: 28.7s	remaining: 11.2s
360:	total: 28.8s	remaining: 11.1s
361:	total: 28.8s	remaining: 11s
362:	total: 28.9s	remaining: 10.9s
363:	total: 29s	remaining: 10.8s
364:	total: 29s	remaining: 10.7s

76:	total: 5.65s	remaining: 31s
77:	total: 5.71s	remaining: 30.9s
78:	total: 5.76s	remaining: 30.7s
79:	total: 5.82s	remaining: 30.6s
80:	total: 5.9s	remaining: 30.5s
81:	total: 5.96s	remaining: 30.4s
82:	total: 6.03s	remaining: 30.3s
83:	total: 6.11s	remaining: 30.3s
84:	total: 6.19s	remaining: 30.2s
85:	total: 6.26s	remaining: 30.2s
86:	total: 6.33s	remaining: 30.1s
87:	total: 6.38s	remaining: 29.9s
88:	total: 6.44s	remaining: 29.7s
89:	total: 6.49s	remaining: 29.6s
90:	total: 6.54s	remaining: 29.4s
91:	total: 6.59s	remaining: 29.2s
92:	total: 6.66s	remaining: 29.1s
93:	total: 6.71s	remaining: 29s
94:	total: 6.77s	remaining: 28.9s
95:	total: 6.83s	remaining: 28.7s
96:	total: 6.88s	remaining: 28.6s
97:	total: 6.93s	remaining: 28.4s
98:	total: 6.99s	remaining: 28.3s
99:	total: 7.05s	remaining: 28.2s
100:	total: 7.12s	remaining: 28.1s
101:	total: 7.2s	remaining: 28.1s
102:	total: 7.29s	remaining: 28.1s
103:	total: 7.37s	remaining: 28.1s
104:	total: 7.45s	remaining: 28s
105:	total: 7.51s

316:	total: 23.8s	remaining: 13.7s
317:	total: 23.9s	remaining: 13.7s
318:	total: 24s	remaining: 13.6s
319:	total: 24s	remaining: 13.5s
320:	total: 24.1s	remaining: 13.4s
321:	total: 24.2s	remaining: 13.4s
322:	total: 24.2s	remaining: 13.3s
323:	total: 24.4s	remaining: 13.2s
324:	total: 24.5s	remaining: 13.2s
325:	total: 24.6s	remaining: 13.1s
326:	total: 24.7s	remaining: 13.1s
327:	total: 24.8s	remaining: 13s
328:	total: 24.8s	remaining: 12.9s
329:	total: 24.9s	remaining: 12.8s
330:	total: 24.9s	remaining: 12.7s
331:	total: 25s	remaining: 12.6s
332:	total: 25s	remaining: 12.6s
333:	total: 25.1s	remaining: 12.5s
334:	total: 25.2s	remaining: 12.4s
335:	total: 25.2s	remaining: 12.3s
336:	total: 25.3s	remaining: 12.2s
337:	total: 25.4s	remaining: 12.2s
338:	total: 25.5s	remaining: 12.1s
339:	total: 25.6s	remaining: 12s
340:	total: 25.6s	remaining: 12s
341:	total: 25.7s	remaining: 11.9s
342:	total: 25.8s	remaining: 11.8s
343:	total: 25.8s	remaining: 11.7s
344:	total: 25.9s	remaining: 11.6s

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x172fb0a10>,
             param_grid={'depth': [4, 6, 8], 'eval_metric': ['AUC'],
                         'iterations': [100, 200, 300, 400, 500],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'loss_function': ['MultiClass']})

In [10]:
best_params = grid_search.best_params_
print("Best parameters:", best_params)
best_score = grid_search.best_score_
print("Best training score:", best_score)

best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Best test score:", test_score)

Best parameters: {'depth': 8, 'eval_metric': 'AUC', 'iterations': 500, 'learning_rate': 0.05, 'loss_function': 'MultiClass'}
Best training score: 0.6658823529411764
Best test score: 0.6866666666666666
